## Paquetes Necesarios

In [3]:
from __future__ import annotations

import glob
import os
from typing import Dict, Tuple, List, Optional

import numpy as np
import pandas as pd
from scipy.stats import norm, kstest, kendalltau, t as tdist

from statsmodels.distributions.copula.api import (
    GumbelCopula,
    StudentTCopula,
)

import json
from scipy.stats import lognorm, genpareto, burr12

df = pd.read_csv('../data/clean/datos_limpios_log.csv')

## Cargar modelos marginales

In [4]:
def cargar_cola(name: str, carpeta: str) -> Dict:
    pattern = os.path.join(carpeta, f"{name}_tail_*.csv")
    files = glob.glob(pattern)
    if not files:
        raise FileNotFoundError(f"No se encontró tail para {name} en {carpeta}")

    ruta = files[0]
    dfp = pd.read_csv(ruta)

    if dfp.empty:
        raise ValueError(f"CSV de tail vacío: {ruta}")

    model = str(dfp.loc[0, "model_name"]).strip()
    u = float(dfp.loc[0, "u_opt"])
    p_u = float(dfp.loc[0, "p_u"])

    param_cols = [c for c in dfp.columns if "param" in c.lower()]
    param_cols = sorted(param_cols,
                        key=lambda s: int("".join(filter(str.isdigit, s)) or 0))
    params = tuple(float(dfp.loc[0, c]) for c in param_cols)

    return {"model": model, "u": u, "p_u": p_u, "params": params}

## Obtener CDF

In [5]:
def F_exc_tail(y: float, model: str, params: Tuple[float, ...]) -> float:

    if y <= 0:
        return 0.0

    m = model.lower()

    if m == "gpd":
        sigma, xi = params
        if abs(xi) < 1e-12:
            return 1.0 - np.exp(-y / sigma)
        base = 1.0 + xi * y / sigma
        if base <= 0:
            return 1.0
        return 1.0 - base ** (-1.0 / xi)

    if m == "pareto":
        xm, k = params
        return 1.0 - (xm / (xm + y)) ** k

    if m == "burr":
        c, k, lam = params
        return 1.0 - (1.0 + (y / lam) ** c) ** (-k)

    if m.startswith("ln") or m.startswith("lognormal"):
        mu, sig = params
        return float(norm.cdf((np.log(y) - mu) / sig))

    raise ValueError(f"Modelo de cola no implementado en F_exc_tail: {model}")


def F_hybrid(x: float,
             body_sample: np.ndarray,
             tail: Dict,
             a: float = 1.0,
             b: float = 2.0) -> float:

    u = tail["u"]
    p_u = tail["p_u"]

    if x <= u:
        sample = np.asarray(body_sample)
        sample = sample[~np.isnan(sample)]
        if sample.size == 0:
            return np.nan
        s = np.sort(sample)
        r = np.searchsorted(s, x, side="right")
        return (r + a) / (len(s) + b)

    y = x - u
    Fexc = F_exc_tail(y, tail["model"], tail["params"])
    return 1.0 - p_u * (1.0 - Fexc)

## Pasar a Unif con prueba KS

In [6]:
def ks_test_uniform(U: np.ndarray) -> Tuple[float, float]:

    stat, pval = kstest(U, "uniform")
    return float(stat), float(pval)

In [7]:
def U_variable(series: pd.Series,
               name: str,
               carpeta_tail: str,
               clip_eps: float = 1e-6,
               use_empirical_if_ks_fails: bool = True,
               alpha_ks: float = 0.05
               ) -> Tuple[np.ndarray, np.ndarray, Dict, float, float]:

    tail = cargar_cola(name, carpeta_tail)

    vals = series.to_numpy(dtype=float)
    vals = vals.astype(float)
    body_sample = vals[vals <= tail["u"]]

    # --- 1) Transformación híbrida (KDE+cola EVT) ---
    U = np.array(
        [F_hybrid(float(x), body_sample, tail) for x in vals],
        dtype=float
    )
    U = np.clip(U, clip_eps, 1.0 - clip_eps)

    ks_stat, ks_p = ks_test_uniform(U)

    # --- 2) Si el KS "falla", usar distribución empírica ---
    if use_empirical_if_ks_fails and np.isfinite(ks_p) and ks_p < alpha_ks:
        # CDF empírica: \tilde u_j = rank(x_j)/(n+1)
        sample = vals[np.isfinite(vals)]
        s = np.sort(sample)
        ranks = np.searchsorted(s, vals, side="right")  # número de <= x_j
        U = ranks / (len(s) + 1.0)
        U = np.clip(U, clip_eps, 1.0 - clip_eps)

        # Opcional: recalcular KS solo por curiosidad
        ks_stat, ks_p = ks_test_uniform(U)

    return U, body_sample, tail, ks_stat, ks_p


## Ajuste de Cópulas

In [8]:
def fit_copulas(U1: np.ndarray,
                U2: np.ndarray) -> Dict:

    U1 = np.asarray(U1, float)
    U2 = np.asarray(U2, float)
    mask = np.isfinite(U1) & np.isfinite(U2)
    data = np.column_stack([U1[mask], U2[mask]])

    if data.shape[0] < 5:
        raise ValueError("Muy pocos datos para ajustar cópulas")

    # ============================
    # 1) Kendall tau (una sola vez)
    # ============================
    tau, _ = kendalltau(data[:, 0], data[:, 1])
    if not np.isfinite(tau):
        tau = 0.0

    # ============================
    # 2) Gumbel
    # ============================
    # Puedes usar la relación tau = 1 - 1/theta,
    # o usar fit_corr_param. Dejo la versión sencilla con fórmula:
    tau_g = np.clip(tau, 1e-6, 0.999)
    theta = 1.0 / (1.0 - tau_g)  # θ de Gumbel
    gcop = GumbelCopula(theta=theta)

    ll_g = float(np.sum(gcop.logpdf(data)))
    aic_g = -2.0 * ll_g + 2.0 * 1  # k=1

    # ============================
    # 3) t-Student
    # ============================
    # statsmodels NO tiene .fit(), solo fit_corr_param.
    # 1) fijamos un df (nu) o luego haces búsqueda en rejilla
    nu = 4.0  # por ejemplo, puedes cambiar esto

    # 2) estimamos la correlación a partir de los datos
    tcop_tmp = StudentTCopula(df=nu)      # corr=None al inicio
    rho_hat = float(tcop_tmp.fit_corr_param(data))  # escalar en k_dim=2
    rho_hat = np.clip(rho_hat, -0.999, 0.999)


    # Matriz de correlación inicial
    R = np.array([[1.0, rho_hat],
                [rho_hat, 1.0]], dtype=float)

    # Regularización para evitar problemas numéricos
    eps = 1e-6
    R_reg = (1 - eps) * R + eps * np.eye(2)

    # Crear la cópula t con matriz regularizada
    tcop = StudentTCopula(corr=R_reg, df=nu)


    ll_t = float(np.sum(tcop.logpdf(data)))
    aic_t = -2.0 * ll_t + 2.0 * 2  # parámetros libres: rho y nu (aunque aquí nu está fijado)

    # ============================
    # 4) Modelo ganador
    # ============================
    winner = "t-student" if aic_t < aic_g else "gumbel"

    return {
        "tau": float(tau),
        "gumbel": {
            "theta": float(theta),
            "ll": ll_g,
            "aic": aic_g
        },
        "t": {
            "R": R,
            "nu": float(nu),
            "ll": ll_t,
            "aic": aic_t
        },
        "winner": winner
    }


## Simular la Cola

In [9]:
def simulate_copula(fit: Dict,
                    n_sims: int = 50_000,
                    random_state: Optional[int] = None) -> np.ndarray:

    rng = np.random.default_rng(random_state)

    if fit["winner"] == "gumbel":
        theta = fit["gumbel"]["theta"]
        cop = GumbelCopula(theta=theta)
        U = cop.rvs(nobs=n_sims, random_state=random_state)
        U = np.asarray(U, float)

    else:
        R = np.asarray(fit["t"]["R"], float)
        nu = float(fit["t"]["nu"])
        k = R.shape[0]

        L = np.linalg.cholesky(R + 1e-12 * np.eye(k))

        g = rng.standard_normal(size=(n_sims, k))
        z = g @ L.T

        w = rng.chisquare(df=nu, size=n_sims) / nu
        t_samples = z / np.sqrt(w[:, None])

        U = tdist.cdf(t_samples, df=nu)

    return np.clip(U, 1e-12, 1.0 - 1e-12)


## Cuantil

In [10]:
def Q_hybrid(alpha: float,
             body_sample: np.ndarray,
             tail: Dict) -> float:

    alpha = float(np.clip(alpha, 1e-12, 1.0 - 1e-12))
    u = tail["u"]
    p_u = tail["p_u"]

    sample = np.asarray(body_sample)
    sample = sample[~np.isnan(sample)]

    if alpha <= (1.0 - p_u) or sample.size == 0:
        return float(np.quantile(sample, alpha))

    alpha_exc = (alpha - (1.0 - p_u)) / p_u
    alpha_exc = float(np.clip(alpha_exc, 1e-12, 1.0 - 1e-12))

    m = tail["model"].lower()
    params = tail["params"]

    if m == "gpd":
        sigma, xi = params
        if abs(xi) < 1e-12:
            q_exc = -sigma * np.log1p(-alpha_exc)
        else:
            q_exc = (sigma / xi) * (np.power(1.0 - alpha_exc, -xi) - 1.0)

    elif m == "pareto":
        xm, k = params
        q_exc = xm * (np.power(1.0 - alpha_exc, -1.0 / k) - 1.0)

    elif m == "burr":
        c, k, lam = params
        inner = np.power(1.0 - alpha_exc, -1.0 / k) - 1.0
        inner = max(inner, 1e-18)
        q_exc = lam * (inner ** (1.0 / c))

    elif m.startswith("ln") or m.startswith("lognormal"):
        mu, sig = params
        z = norm.ppf(alpha_exc)
        q_exc = np.exp(mu + sig * z)

    else:
        raise ValueError(f"Modelo no implementado en Q_hybrid: {tail['model']}")

    return float(u + q_exc)

## VaR y CVaR para parejas

In [11]:
def simulate_joint_losses(U_sim: np.ndarray,
                          body1: np.ndarray,
                          tail1: Dict,
                          body2: np.ndarray,
                          tail2: Dict) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Recibe:
        U_sim: matriz (N x 2) de uniformes conjuntas (salida de simulate_copula)
        body1, tail1: muestra de cuerpo + cola para la variable 1
        body2, tail2: idem para la variable 2

    Devuelve:
        X1, X2, S = X1 + X2
    """
    U1 = U_sim[:, 0]
    U2 = U_sim[:, 1]

    X1 = np.array([Q_hybrid(float(a), body1, tail1) for a in U1], dtype=float)
    X2 = np.array([Q_hybrid(float(a), body2, tail2) for a in U2], dtype=float)
    S = X1 + X2
    return X1, X2, S


# ============================================================
# 8. VAR Y CVAR DE UNA SERIE
# ============================================================

def var_cvar(S: np.ndarray, alpha: float) -> Tuple[float, float]:
    """
    Calcula VaR y CVaR (TVaR) empíricos para una muestra S.

    VaR = cuantil alpha
    CVaR = media condicional S > VaR
    """
    S = np.asarray(S, float)
    S = S[np.isfinite(S)]

    if S.size == 0:
        return np.nan, np.nan

    q = float(np.quantile(S, alpha))
    tail = S[S > q]
    cvar = float(tail.mean()) if tail.size > 0 else q
    return q, cvar

## Pipe completo

In [12]:
def dependencia_y_riesgo(df: pd.DataFrame,
                         col1: str,
                         val1: str,
                         carpeta1: str,
                         col2: str,
                         val2: str,
                         carpeta2: str,
                         n_sims: int = 50_000,
                         alphas: Tuple[float, ...] = (0.95, 0.99),
                         random_state: Optional[int] = None) -> Optional[Dict]:
    """
    Pipeline completo para una pareja (val1 de col1) vs (val2 de col2):

    1) filtra df por col1 == val1 y col2 == val2
    2) obtiene U1, U2 via F_hybrid + cola EVT
    3) KS test de uniformidad para U1 y U2
    4) ajusta cópulas Gumbel y t-Student y elige la mejor
    5) simula U_sim ~ copula ganadora
    6) aplica Q_hybrid para obtener X1, X2 y S = X1+X2
    7) calcula VaR y CVaR para cada alpha en alphas

    Devuelve un diccionario con resultados o None si no hay datos suficientes.
    """
    df_pair = df[(df[col1] == val1) & (df[col2] == val2)].copy()
    if df_pair.shape[0] < 20:
        # muy pocos datos
        return None

    # Marginal 1
    U1, body1, tail1, ks1_stat, ks1_p = U_variable(
        df_pair["total"], name=str(val1), carpeta_tail=carpeta1,
        use_empirical_if_ks_fails=True, alpha_ks=0.05
    )

    U2, body2, tail2, ks2_stat, ks2_p = U_variable(
        df_pair["total"], name=str(val2), carpeta_tail=carpeta2,
        use_empirical_if_ks_fails=True, alpha_ks=0.05
    )

    # Ajuste de cópulas
    fit = fit_copulas(U1, U2)

    # Simulación conjunta en espacio U
    U_sim = simulate_copula(fit, n_sims=n_sims, random_state=random_state)

    # Simulación de pérdidas
    X1, X2, S = simulate_joint_losses(U_sim, body1, tail1, body2, tail2)

    res = {
        "col1": col1,
        "val1": val1,
        "col2": col2,
        "val2": val2,
        "n_obs": int(df_pair.shape[0]),
        "ks1_stat": ks1_stat,
        "ks1_p": ks1_p,
        "ks2_stat": ks2_stat,
        "ks2_p": ks2_p,
        "tau": fit["tau"],
        "best_copula": fit["winner"],
        "gumbel_aic": fit["gumbel"]["aic"],
        "t_aic": fit["t"]["aic"],
    }

    for a in alphas:
        VaR, CVaR_ = var_cvar(S, a)
        res[f"VaR({a})"] = VaR
        res[f"CVaR({a})"] = CVaR_

    return res

In [13]:
def todas_dependencias(
    df: pd.DataFrame,
    provincias: List[str],
    categorias: List[str],
    sectores: List[str],
    carpeta_prov: str = "../res/provincia",
    carpeta_cat: str = "../res/categoria",
    carpeta_sec: str = "../res/sector",
    n_sims: int = 50_000,
    alphas: Tuple[float, ...] = (0.95, 0.99),
    random_state: Optional[int] = 42,
) -> pd.DataFrame:
    """
    Ejecuta dependencia_y_riesgo para:

        1) provincia - sector
        2) provincia - categoria
        3) categoria - sector

    y devuelve un DataFrame con todos los resultados.
    """
    resultados = []

    # 1) provincia - sector
    for p in provincias:
        for s in sectores:
            r = dependencia_y_riesgo(
                df, "provincia", p, carpeta_prov,
                "sector", s, carpeta_sec,
                n_sims=n_sims, alphas=alphas, random_state=random_state
            )
            if r is not None:
                r["tipo_par"] = "provincia-sector"
                resultados.append(r)

    # 2) provincia - categoria
    for p in provincias:
        for c in categorias:
            r = dependencia_y_riesgo(
                df, "provincia", p, carpeta_prov,
                "categoria", c, carpeta_cat,
                n_sims=n_sims, alphas=alphas, random_state=random_state
            )
            if r is not None:
                r["tipo_par"] = "provincia-categoria"
                resultados.append(r)

    # 3) categoria - sector
    for c in categorias:
        for s in sectores:
            r = dependencia_y_riesgo(
                df, "categoria", c, carpeta_cat,
                "sector", s, carpeta_sec,
                n_sims=n_sims, alphas=alphas, random_state=random_state
            )
            if r is not None:
                r["tipo_par"] = "categoria-sector"
                resultados.append(r)

    if not resultados:
        return pd.DataFrame()

    return pd.DataFrame(resultados)

# Uso

In [14]:
# Supongamos que df ya está cargado y limpio
provincias = sorted(df["provincia"].dropna().unique())
categorias = sorted(df["categoria"].dropna().unique())
sectores   = sorted(df["sector"].dropna().unique())

tabla = todas_dependencias(
    df,
    provincias=provincias,
    categorias=categorias,
    sectores=sectores,
    carpeta_prov="../res/provincias",
    carpeta_cat="../res/categorias",
    carpeta_sec="../res/sectores",
    n_sims=200_000,
    alphas=(0.95, 0.99),
    random_state=123
)


c:\Users\andre\anaconda3\envs\Proyectos\Lib\site-packages\statsmodels\distributions\copula\transforms.py:139: RuntimeWarning: overflow encountered in power
  return np.power(-np.log(t), theta)
c:\Users\andre\anaconda3\envs\Proyectos\Lib\site-packages\statsmodels\distributions\copula\transforms.py:145: RuntimeWarning: overflow encountered in power
  return - theta * (-np.log(t))**(theta - 1) / t
c:\Users\andre\anaconda3\envs\Proyectos\Lib\site-packages\statsmodels\distributions\copula\archimedean.py:187: RuntimeWarning: divide by zero encountered in log
  logpdfv = np.sum(np.log(np.abs(phi_d1(u, *args))), axis)
c:\Users\andre\anaconda3\envs\Proyectos\Lib\site-packages\statsmodels\distributions\copula\transforms.py:158: RuntimeWarning: overflow encountered in square
  d2 = (phi**(2 / th) + (th - 1) * phi**(1 / th)) / (phi**2 * th**2)
c:\Users\andre\anaconda3\envs\Proyectos\Lib\site-packages\statsmodels\distributions\copula\transforms.py:158: RuntimeWarning: invalid value encountered in d

KeyboardInterrupt: 

In [ ]:
print(tabla)

         col1                val1       col2                val2  n_obs  \
0   provincia            Alajuela     sector             HÍDRICO     29   
1   provincia            Alajuela     sector     INFRAESTRUCTURA    174   
2   provincia            Alajuela     sector               OTROS     54   
3   provincia            Alajuela     sector          PRODUCTIVO     47   
4   provincia            Alajuela     sector              SOCIAL     59   
5   provincia             Cartago     sector     INFRAESTRUCTURA     70   
6   provincia             Cartago     sector               OTROS     28   
7   provincia             Cartago     sector          PRODUCTIVO     22   
8   provincia             Cartago     sector              SOCIAL     41   
9   provincia          Guanacaste     sector             HÍDRICO     39   
10  provincia          Guanacaste     sector     INFRAESTRUCTURA    190   
11  provincia          Guanacaste     sector               OTROS     53   
12  provincia          Gu

In [ ]:
df

,Unnamed: 0,ano,categoria,provincia,canton,latitud,longitud,total,sector
0,1,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,16.842111,INFRAESTRUCTURA
1,2,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,20.100323,SOCIAL
2,3,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,15.641758,OTROS
3,4,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,15.509367,OTROS
4,5,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,17.298166,SOCIAL
...,...,...,...,...,...,...,...,...,...
2235,2487,2022,Hidrometereológico,San José,Pérez Zeledón,9.384718038,-83.706217,19.902285,INFRAESTRUCTURA
2236,2488,2022,Hidrometereológico,San José,Pérez Zeledón,9.384718038,-83.706217,18.545246,INFRAESTRUCTURA
2237,2489,2022,Hidrometereológico,San José,Pérez Zeledón,9.384718038,-83.706217,20.617905,OTROS
2238,2490,2022,Hidrometereológico,San José,Pérez Zeledón,9.384718038,-83.706217,21.428111,HÍDRICO


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/clean/datos_limpios_log.csv")

# Eliminamos valores absurdos o NaN en total
df = df[df["total"].notna()]

# Z-scores por dominio
df["sev_prov"] = df.groupby("provincia")["total"].transform(lambda x: (x - x.mean()) / x.std())
df["sev_sector"] = df.groupby("sector")["total"].transform(lambda x: (x - x.mean()) / x.std())
df["sev_cat"] = df.groupby("categoria")["total"].transform(lambda x: (x - x.mean()) / x.std())

# Reemplazar NaN (si una provincia o sector tiene 1 observación)
df[["sev_prov", "sev_sector", "sev_cat"]] = df[["sev_prov", "sev_sector", "sev_cat"]].fillna(0.0)

df.head()


,Unnamed: 0,ano,categoria,provincia,canton,latitud,longitud,total,sector,sev_prov,sev_sector,sev_cat
0,1,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,16.842111,INFRAESTRUCTURA,-0.847448,-1.279643,-1.062868
1,2,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,20.100323,SOCIAL,0.834937,0.987651,0.666613
2,3,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,15.641758,OTROS,-1.467253,-1.242265,-1.700024
3,4,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,15.509367,OTROS,-1.535613,-1.298810,-1.770298
4,5,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,17.298166,SOCIAL,-0.611963,-0.559420,-0.820791


In [ ]:
df

,Unnamed: 0,ano,categoria,provincia,canton,latitud,longitud,total,sector,sev_prov,sev_sector,sev_cat
0,1,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,16.842111,INFRAESTRUCTURA,-0.847448,-1.279643,-1.062868
1,2,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,20.100323,SOCIAL,0.834937,0.987651,0.666613
2,3,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,15.641758,OTROS,-1.467253,-1.242265,-1.700024
3,4,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,15.509367,OTROS,-1.535613,-1.298810,-1.770298
4,5,2005,Hidrometereológico,Cartago,Alvarado,9.91708244,-83.809449,17.298166,SOCIAL,-0.611963,-0.559420,-0.820791
...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2487,2022,Hidrometereológico,San José,Pérez Zeledón,9.384718038,-83.706217,19.902285,INFRAESTRUCTURA,0.600717,0.401760,0.561493
2236,2488,2022,Hidrometereológico,San José,Pérez Zeledón,9.384718038,-83.706217,18.545246,INFRAESTRUCTURA,-0.162791,-0.343861,-0.158832
2237,2489,2022,Hidrometereológico,San José,Pérez Zeledón,9.384718038,-83.706217,20.617905,OTROS,1.003344,0.883073,0.941349
2238,2490,2022,Hidrometereológico,San José,Pérez Zeledón,9.384718038,-83.706217,21.428111,HÍDRICO,1.459188,1.635893,1.371412


## Arreglos

In [ ]:
print('Años y descripción')
print(df['ano'].unique())
print(df['ano'].describe())
print('PRovincias y descripción')
print(df['provincia'].unique())
print(df['provincia'].describe())
print('Categorias y descripción')
print(df['categoria'].unique())
print(df['categoria'].describe())
print('Sectores y descripción')
print(df['sector'].unique())
print(df['sector'].describe())

Años y descripción
[2005 2006 2007 2008 2009 2010 2012 2014 2015 2016 2017 2019 2020 2021
 2022]
count    2240.000000
mean     2012.076339
std         5.418936
min      2005.000000
25%      2007.000000
50%      2010.000000
75%      2017.000000
max      2022.000000
Name: ano, dtype: float64
PRovincias y descripción
['Cartago' 'Heredia' 'Limón' 'Alajuela' 'Guanacaste' 'Puntarenas'
 'San José']
count         2240
unique           7
top       San José
freq           469
Name: provincia, dtype: object
Categorias y descripción
['Hidrometereológico' 'Geológico']
count                   2240
unique                     2
top       Hidrometereológico
freq                    2102
Name: categoria, dtype: object
Sectores y descripción
['INFRAESTRUCTURA' 'SOCIAL' 'OTROS' 'HÍDRICO' 'PRODUCTIVO']
count                2240
unique                  5
top       INFRAESTRUCTURA
freq                  990
Name: sector, dtype: object


In [ ]:
import pandas as pd

group_cols = ["provincia", "categoria"]

resultados_por_grupo = {}

for (prov, cat), g in df.groupby(group_cols):

    # Pivot: una fila por año, columnas = sectores, valores = total
    df_wide = g.pivot_table(
        index="ano",          # aquí está el “por año”
        columns="sector",
        values="total",
        aggfunc="sum"
    )

    df_wide.columns.name = None
    # df_wide.index son los años; si quieres columna explícita:
    df_wide = df_wide.reset_index()  # ahora tienes una columna 'ano'

    # Seleccionar sólo las columnas de totales (todas menos 'ano')
    cols_totales = [c for c in df_wide.columns if c != "ano"]
    X = df_wide[cols_totales].dropna()  # matriz n×d para la cópula

    # Guardar por si quieres inspeccionar luego
    resultados_por_grupo[(prov, cat)] = {
        "df_wide": df_wide,
        "X": X
    }


## Cargar las marginales

In [ ]:

def cargar_marginal(path):

    with open(os.path.join(path, "metadata.json"), "r") as f:
        meta = json.load(f)

    u_opt = meta["u_opt"]
    p_u   = meta["p_u"]

    # --- cargar cuerpo ---
    body = meta["body"]
    body_model = body["model"]
    body_type  = body["type"]

    if body_type == "parametric":
        params = body["params"]
        if body_model == "Lognormal":
            F_body = lambda x: lognorm.cdf(x, s=params[1], scale=np.exp(params[0]))
        elif body_model == "Gamma":
            a,b = params
            F_body = lambda x: stats.gamma.cdf(x, a=a, scale=1/b)
        elif body_model == "Weibull":
            k, lam = params
            F_body = lambda x: stats.weibull_min.cdf(x, c=k, scale=lam)
        else:
            raise ValueError("Modelo de cuerpo no implementado.")

    else: # KDE
        df_kde = pd.read_csv(os.path.join(path, body["csv"]))
        xs = df_kde["x"].to_numpy()
        fs = df_kde["f"].to_numpy()
        cdf_vals = np.cumsum(fs)/np.sum(fs)
        F_body = lambda x: np.interp(x, xs, cdf_vals)

    # --- cargar cola ---
    tail = meta["tail"]
    if tail["model"] is None:
        F_tail = None
    else:
        tail_model = tail["model"]
        tail_type  = tail["type"]
        if tail_type == "parametric":
            params = tail["params"]
            if tail_model == "GPD":
                xi, sg = params
                F_tail = lambda y: genpareto.cdf(y, c=xi, scale=sg)
            elif tail_model == "Burr":
                c,k,lam = params
                F_tail = lambda y: burr12.cdf(y, c=c, d=k, scale=lam)
            elif tail_model == "Pareto":
                a,y0 = params
                F_tail = lambda y: 1 - (y0/(y+y0))**a
            else:
                raise ValueError("Modelo de cola no implementado.")
        else: # KDE cola
            df_kde = pd.read_csv(os.path.join(path, tail["csv"]))
            xs = df_kde["x"].to_numpy()
            fs = df_kde["f"].to_numpy()
            cdf_vals = np.cumsum(fs)/np.sum(fs)
            F_tail = lambda y: np.interp(y, xs, cdf_vals)

    # --- CDF híbrida completa ---
    def F(x):
        x = np.asarray(x)
        out = np.zeros_like(x)

        mask_body = x <= u_opt
        mask_tail = x > u_opt

        out[mask_body] = (1 - p_u) * F_body(x[mask_body])
        out[mask_tail] = (1 - p_u) + p_u * F_tail(x[mask_tail] - u_opt)

        return out

    return F


In [ ]:
def transformar_a_u(x, F):
    x = np.asarray(x)
    u = F(x)
    eps = 1e-6
    u = np.clip(u, eps, 1 - eps)
    return u


In [ ]:
from scipy.stats import kstest

def verificar_uniformidad(u, alpha=0.05):
    """
    Test KS para ver si u ~ U(0,1).
    Devuelve (es_uniforme, pvalor).
    """
    stat, p = kstest(u, 'uniform')
    return p > alpha, p


In [ ]:
def forzar_u_empirico(u):
    """
    Genera U(0,1) empírica usando ranking:
    rank(u) / (n+1)  -> uniforme perfecta
    """
    u = np.asarray(u)
    r = u.argsort().argsort() + 1  # ranking 1...n
    n = len(u)
    return r / (n + 1)


In [ ]:
def generar_u(x, F, alpha=0.05):
    """
    1. Genera U = F(x)
    2. Verifica si U ~ U(0,1)
    3. Si NO lo cumple, genera U empírica
    """
    u = transformar_a_u(x, F)
    
    es_uni, p = verificar_uniformidad(u, alpha)
    
    if es_uni:
        print(f"[OK] KS no rechaza uniformidad (p={p:.4f}).")
        return u
    else:
        print(f"[ALERTA] KS rechaza uniformidad (p={p:.4f}). Se usa U empírica.")
        return forzar_u_empirico(u)


In [ ]:
from statsmodels.distributions.copula.api import (
    GaussianCopula,
    StudentTCopula,
    GumbelCopula
)

def ajustar_copulas(u1, u2):
    U = np.column_stack([u1, u2])  # matriz Nx2

    resultados = {}

    # Gaussiana
    gauss = GaussianCopula()
    pars_g = gauss.fit_corr_param(U)
    ll_g = gauss.loglikelihood(pars_g)
    resultados["Gaussian"] = (gauss, pars_g, ll_g)

    # t-Student
    tcop = StudentTCopula()
    pars_t = tcop.fit_corr_param(U)
    ll_t = tcop.loglikelihood(pars_t)
    resultados["t"] = (tcop, pars_t, ll_t)

    # Gumbel
    gum = GumbelCopula()
    pars_gum = gum.fit_corr_param(U)
    ll_gum = gum.loglikelihood(pars_gum)
    resultados["Gumbel"] = (gum, pars_gum, ll_gum)

    return resultados


In [ ]:
def mejor_copula(resultados):
    ll = {k: v[2] for k, v in resultados.items()}
    best = max(ll, key=ll.get)
    return best, resultados[best]


In [ ]:
def simular_copula(copula, params, n=50000):
    U_sim = copula.random(params, n)
    return U_sim


In [ ]:
def construir_Q_body(path, body_meta):
    if body_meta["type"] == "parametric":
        params = body_meta["params"]
        model = body_meta["model"]

        if model == "Lognormal":
            mu,sigma = params
            return lambda u: lognorm.ppf(u, s=sigma, scale=np.exp(mu))

        elif model == "Gamma":
            a,b = params
            return lambda u: stats.gamma.ppf(u, a=a, scale=1/b)

        elif model == "Weibull":
            k,lam = params
            return lambda u: stats.weibull_min.ppf(u, c=k, scale=lam)

    else:
        # KDE body: invertir tabla x vs cdf
        df_kde = pd.read_csv(os.path.join(path, body_meta["csv"]))
        xs = df_kde["x"].to_numpy()
        fs = df_kde["f"].to_numpy()
        cdf_vals = np.cumsum(fs)/np.sum(fs)
        return lambda u: np.interp(u, cdf_vals, xs)


In [ ]:
def construir_Q_tail(path, tail_meta):
    if tail_meta["model"] is None:
        return None

    if tail_meta["type"] == "parametric":
        params = tail_meta["params"]
        model = tail_meta["model"]

        if model == "GPD":
            xi, sg = params
            return lambda u: genpareto.ppf(u, c=xi, scale=sg)

        elif model == "Burr":
            c,k,lam = params
            return lambda u: burr12.ppf(u, c=c, d=k, scale=lam)

        elif model == "Pareto":
            a,y0 = params
            return lambda u: y0*((1-u)**(-1/a) - 1)

    else:
        df_kde = pd.read_csv(os.path.join(path, tail_meta["csv"]))
        xs = df_kde["x"].to_numpy()
        fs = df_kde["f"].to_numpy()
        cdf_vals = np.cumsum(fs)/np.sum(fs)
        return lambda u: np.interp(u, cdf_vals, xs)


In [ ]:
def cargar_inversa(path):
    with open(os.path.join(path, "metadata.json"), "r") as f:
        meta = json.load(f)

    u_opt = meta["u_opt"]
    p_u   = meta["p_u"]
    body  = meta["body"]
    tail  = meta["tail"]

    Qb = construir_Q_body(path, body)
    Qt = construir_Q_tail(path, tail)

    def Q(u):
        u = np.asarray(u)
        out = np.zeros_like(u)

        # región cuerpo: u <= 1 - p_u
        mask_body = u <= (1 - p_u)
        mask_tail = u > (1 - p_u)

        # convertir escala:
        out[mask_body] = Qb(u[mask_body] / (1 - p_u))
        out[mask_tail] = u_opt + Qt((u[mask_tail] - (1 - p_u)) / p_u)

        return out

    return Q


In [ ]:
def transformar_simulaciones(U_sim, Q1, Q2):
    X1 = Q1(U_sim[:, 0])
    X2 = Q2(U_sim[:, 1])
    return np.column_stack([X1, X2])


In [ ]:
def VaR(X, alpha=0.95):
    return np.quantile(X, alpha)

def CVaR(X, alpha=0.95):
    var = VaR(X, alpha)
    return X[X > var].mean()


In [ ]:
path1 = "res/marginales/combinaciones/Heredia_INFRAESTRUCTURA"
path2 = "res/marginales/combinaciones/Heredia_SOCIAL"

F1 = cargar_marginal(path1)
F2 = cargar_marginal(path2)

Q1 = cargar_inversa(path1)
Q2 = cargar_inversa(path2)

x1 = df[(df.provincia=="Heredia") & (df.sector=="INFRAESTRUCTURA")]["total"]
x2 = df[(df.provincia=="Heredia") & (df.sector=="SOCIAL")]["total"]

u1 = generar_u(x1, F1)
u2 = generar_u(x2, F2)

res = ajustar_copulas(u1, u2)
best_name, (best_copula, best_params, ll) = mejor_copula(res)

print("La mejor cópula es:", best_name)

U_sim = simular_copula(best_copula, best_params, n=50000)


X_sim = transformar_simulaciones(U_sim, Q1, Q2)

var95 = VaR(X_sim[:,0], 0.95)
cvar95 = CVaR(X_sim[:,0], 0.95)

print("VaR 95%:", var95)
print("CVaR 95%:", cvar95)



# LIMPIO

In [ ]:
# PRimero se debe obtener la función de dist

def build_cdf(body_info, tail_info, u_opt, p_u):
    body_type = body_info["tipo"]
    tail_type = tail_info["tipo"]

    if body_type == "Parametrico":
        name = body_info["modelo"]
        pars = body_info["parametros"]

        if name == "Gamma":
            cdf_body = lambda x: st.gamma.cdf(x, a=pars[0], scale=1/pars[1])
        elif name == "Lognormal":
            cdf_body = lambda x: st.lognorm.cdf(x, s=pars[1], scale=np.exp(pars[0]))
        elif name == "Weibull":
            cdf_body = lambda x: st.weibull_min.cdf(x, c=pars[0], scale=pars[1])
        elif name == "Fisk":
            cdf_body = lambda x: st.fisk.cdf(x, c=pars[0], scale=pars[1])
        else:
            raise ValueError("Modelo cuerpo no implementado.")

    else:  # KDE
        xgrid = body_info["x"]
        fgrid = body_info["f"]
        cdf_vals = np.cumsum(fgrid)
        cdf_vals /= cdf_vals[-1]

        def cdf_body(x):
            return np.interp(x, xgrid, cdf_vals)

    # --- CDF cola ---
    if tail_type == "Parametrico":
        name = tail_info["modelo"]
        pars = tail_info["parametros"]

        if name == "GPD":
            cdf_tail = lambda y: st.genpareto.cdf(y, c=pars[0], scale=pars[1])
        elif name == "Burr":
            cdf_tail = lambda y: st.burr12.cdf(y, c=pars[0], d=pars[1], scale=pars[2])
        elif name == "Pareto":
            a,y0 = pars
            cdf_tail = lambda y: 1 - (y0/(y+y0))**a
        else:
            raise ValueError("Modelo cola no implementado.")

    else:
        xgrid = tail_info["x"]
        fgrid = tail_info["f"]
        cdf_vals = np.cumsum(fgrid)
        cdf_vals /= cdf_vals[-1]

        def cdf_tail(y):
            return np.interp(y, xgrid, cdf_vals)

    # --- CDF total ---
    def cdf_total(x):
        x = np.asarray(x)
        F = np.zeros_like(x)

        mask_body = x <= u_opt
        mask_tail = x > u_opt

        F[mask_body] = cdf_body(x[mask_body])
        F[mask_tail] = 1 - p_u + p_u * cdf_tail(x[mask_tail] - u_opt)

        return F

    return cdf_total


In [ ]:
def to_uniform(x, F):
    U = F(x)

    # Test KS
    ks_p = st.kstest(U, 'uniform').pvalue

    if ks_p < 0.05:
        print("[Aviso] No uniforme → usando empírica U = rank/n.")
        U = st.rankdata(x) / (len(x)+1)

    return U


In [ ]:
from statsmodels.distributions.copula.api import (
    GaussianCopula,
    StudentTCopula,
    GumbelCopula
)

def fit_copulas(U1, U2):
    data = np.column_stack([U1, U2])

    models = {}
    # Gaussiana
    g = GaussianCopula()
    gfit = g.fit(data)
    models["gaussian"] = (g, gfit)

    # t
    t = StudentTCopula()
    tfit = t.fit(data)
    models["t"] = (t, tfit)

    # Gumbel
    gu = GumbelCopula()
    gufit = gu.fit(data)
    models["gumbel"] = (gu, gufit)

    # escoger mejor por loglik
    best = max(models.items(), key=lambda m: m[1][1].loglike)
    return models, best


In [ ]:
def simulate_copula(model, params, n):
    return model.simulate(n, params=params)


In [ ]:
def inverse_transform(U, F, grid_x):
    # construir F(x) sobre una grilla fina
    Fx = F(grid_x)

    # interpolar inversa
    return np.interp(U, Fx, grid_x)


In [ ]:
def build_pair_dataset(
    df,
    var1, val1,                # 1er filtro  (ej: "provincia", "Cartago")
    var2, val2,                # 2do filtro  (ej: "sector", "INFRAESTRUCTURA")
    marginales,                # diccionario con info de tus marginales guardados
    modo="colas",              # "colas", "completo" o "cuerpo"
    ambos_en_cola=False        # si True: exige X>u_x Y>u_y
):
    """
    Construye dataset bivariado (X,Y) para cópulas.
    
    Parámetros:
    -----------
    df : dataframe original
    var1, val1 : filtro 1 (ejemplo: provincia="Cartago")
    var2, val2 : filtro 2 (ejemplo: sector="INFRAESTRUCTURA")
    
    marginales : dict indexado por nombre_location, 
                 cada uno con:
                    - u_opt
                    - p_u
                    - modelo cuerpo (body)
                    - modelo cola   (tail)
                    - CDF total
                    - F_inv total

    modo:
        "colas"    → usa solo datos en cola
        "completo" → usa todo
        "cuerpo"   → usa solo x <= u
    
    ambos_en_cola:
        False: usa unión: (x > u_x) OR (y > u_y)
        True:  usa intersección: (x > u_x) AND (y > u_y)
    
    Return:
        df_pairs: dataframe con columnas:
            val1, val2, total_1, total_2
        x: array del primer total según modo
        y: array del segundo total según modo
        info_x, info_y: info de marginales para cada variable
    """

    # ===============================
    # 1. Extraer subconjunto crudo
    # ===============================
    df_temp = df[(df[var1] == val1) & (df[var2] == val2)].copy()

    if len(df_temp) == 0:
        print(f"[Aviso] No hay datos para combinación {var1}={val1}, {var2}={val2}.")
        return None, None, None, None, None

    # Renombrar claras las columnas de totales
    df_pairs = df_temp[[var1, var2, "total"]].copy()
    df_pairs = df_pairs.rename(columns={
        "total": "total_raw"
    })

    # ===============================
    # 2. Ubicar marginales ajustadas
    # ===============================
    key1 = f"{var1}_{val1}"
    key2 = f"{var2}_{val2}"

    if key1 not in marginales or key2 not in marginales:
        print(f"[Error] No se encontró marginal para {key1} o {key2}.")
        return None, None, None, None, None

    info_x = marginales[key1]   # contiene u_opt, p_u, body, tail, CDF, F_inv
    info_y = marginales[key2]

    u_x = info_x["u_opt"]
    u_y = info_y["u_opt"]

    # ===============================
    # 3. Definir X, Y con sus totales
    # ===============================
    X = df_pairs["total_raw"].to_numpy()
    Y = df_pairs["total_raw"].to_numpy()  # no correcto aún, corregimos abajo

    # NOTA:
    # En provincia-sector, df original trae "total" para cada fila según el evento.
    # Para ver dependencia, NECESITÁS construir:
    #   X = total por filtro1 (provincia)
    #   Y = total por filtro2 (sector)
    # Pero ambos vienen del MISMO df_temp.
    #
    # Entonces asumimos que df contiene:
    #   total_x (provincia-val1)    y total_y (sector-val2)
    #
    # Si solo tienes una columna total, NECESITAS haber creado "total_provincia" y "total_sector"
    # antes de usar esta función.
    #
    # La versión final asume esas columnas existen:

    if f"total_{var1}" in df_temp.columns and f"total_{var2}" in df_temp.columns:
        df_pairs["total_x"] = df_temp[f"total_{var1}"]
        df_pairs["total_y"] = df_temp[f"total_{var2}"]
    else:
        raise ValueError(
            "Faltan columnas total_var1 y total_var2 en df. "
            "Debe crear columnas total_provincia, total_sector, total_categoria antes."
        )

    X = df_pairs["total_x"].to_numpy()
    Y = df_pairs["total_y"].to_numpy()

    # ===============================
    # 4. Filtrar según modo
    # ===============================
    if modo == "colas":
        if ambos_en_cola:
            mask = (X > u_x) & (Y > u_y)
        else:
            mask = (X > u_x) | (Y > u_y)
    elif modo == "cuerpo":
        mask = (X <= u_x) & (Y <= u_y)
    elif modo == "completo":
        mask = np.ones(len(X), dtype=bool)
    else:
        raise ValueError("modo debe ser 'colas', 'completo' o 'cuerpo'.")

    Xf = X[mask]
    Yf = Y[mask]

    if len(Xf) == 0:
        print(f"[Aviso] No quedaron datos tras aplicar filtro {modo} en {val1}-{val2}.")
        return None, None, None, None, None

    return df_pairs, Xf, Yf, info_x, info_y


In [ ]:
df_pairs, X, Y, info_x, info_y = build_pair_dataset(
    df,
    var1="provincia", val1="Cartago",
    var2="sector",    val2="INFRAESTRUCTURA",
    marginales=marginales_global,
    modo="colas",
    ambos_en_cola=False
)


NameError: name 'marginales_global' is not defined

In [15]:
import numpy as np
import pandas as pd
from scipy.stats import burr12, genpareto, kstest
from sklearn.neighbors import KernelDensity

# --- Helper: reconstruir CDF / sampler desde un CSV de cola --- #
def cargar_marginal_tail(path_csv):
    df = pd.read_csv(path_csv)
    model_name = df.loc[0, "model_name"]

    if df["model_type"].iloc[0] != "tail":
        raise ValueError("Este CSV no es de cola (model_type != 'tail').")

    # Parámetros MCMC paramétricos
    if model_name in ["GPD", "Burr", "Pareto", "Pareto_max", "LN_tail"]:
        # Aquí asumimos que el CSV de MCMC tiene columnas tipo: Burr_param1, Burr_param2, Burr_param3
        param_cols = [c for c in df.columns if "_param" in c]
        # promedio posterior
        theta_hat = df[param_cols].mean().to_numpy()

        def rvs(n, rng=None):
            rng = np.random.default_rng(rng)
            if model_name == "Burr":
                c, k, lam = theta_hat
                return burr12.rvs(c=c, d=k, scale=lam, size=n, random_state=rng)
            elif model_name == "GPD":
                xi, sigma = theta_hat
                return genpareto.rvs(c=xi, scale=sigma, size=n, random_state=rng)
            # Completar con Pareto, Pareto_max, LN_tail según lo que ya tienes
            else:
                raise NotImplementedError("Falta implementar sampler para " + model_name)

        def cdf(x):
            x = np.asarray(x)
            if model_name == "Burr":
                c, k, lam = theta_hat
                return burr12.cdf(x, c=c, d=k, scale=lam)
            elif model_name == "GPD":
                xi, sigma = theta_hat
                return genpareto.cdf(x, c=xi, scale=sigma)
            # Completar igual que en rvs
            else:
                raise NotImplementedError("Falta implementar CDF para " + model_name)

        return rvs, cdf

    # Caso KDE de cola
    elif model_name.startswith("KDE-"):
        xs = df["x"].to_numpy()
        fs = df["f"].to_numpy()
        # normalizar por si acaso
        fs = fs / np.trapz(fs, xs)
        cdf_grid = np.cumsum(fs) / np.sum(fs)

        def rvs(n, rng=None):
            rng = np.random.default_rng(rng)
            u = rng.random(n)
            return np.interp(u, cdf_grid, xs)

        def cdf(x):
            return np.interp(x, xs, cdf_grid, left=0.0, right=1.0)

        return rvs, cdf

    else:
        raise ValueError(f"Modelo de cola no reconocido: {model_name}")

# --- Paso 1 para cópulas: construir U para una combinación --- #
def construir_U_para_copula_tail(archivos_marginales, n_sim=5000, rng=None):
    """
    archivos_marginales: dict con nombre de variable -> path CSV de su marginal de cola
       ej: {
          "provincia": "../res/provincias/Cartago_tail_Burr.csv",
          "sector":    "../res/sectores/INFRAESTRUCTURA_tail_Burr.csv"
       }
    """
    rng = np.random.default_rng(rng)

    U_cols = {}
    X_cols = {}

    for nombre_var, path in archivos_marginales.items():
        rvs, cdf = cargar_marginal_tail(path)

        # 1. Simular X
        x = rvs(n_sim, rng=rng)

        # 2. Transformar a U
        u = cdf(x)

        # 3. Verificar U ~ U(0,1)
        stat, pvalue = kstest(u, 'uniform')
        print(f"[{nombre_var}] KS(U(0,1)) p-valor = {pvalue:.4f}")

        if pvalue < 0.05:
            # U empíricas: ranking/(n+1)
            ranks = np.argsort(np.argsort(x)) + 1
            u_emp = ranks / (n_sim + 1.0)
            U_cols[nombre_var] = u_emp
            X_cols[nombre_var] = x
            print(f"[{nombre_var}] KS rechazó U(0,1). Se usan U empíricas.")
        else:
            U_cols[nombre_var] = u
            X_cols[nombre_var] = x

    U = pd.DataFrame(U_cols)
    X = pd.DataFrame(X_cols)
    return U, X


In [16]:
archivos = {
    "provincia": f"../res/provincias/Cartago_tail_Burr.csv",
    "sector":    f"../res/sectores/INFRAESTRUCTURA_tail_Burr.csv",
}
U, X = construir_U_para_copula_tail(archivos, n_sim=10000, rng=2025)


[provincia] KS(U(0,1)) p-valor = 0.2062
[sector] KS(U(0,1)) p-valor = 0.0873


In [19]:
import numpy as np
from scipy.stats import kendalltau, spearmanr
from statsmodels.distributions.copula.api import (
    GaussianCopula,
    StudentTCopula,
    GumbelCopula
)

# --------------------------------------------------------
# 1. Dependencias empíricas
# --------------------------------------------------------
def dependencia_empirica(U):
    """
    U: dataframe con columnas ["var1", "var2"] en [0,1].
    """
    x = U.iloc[:,0]
    y = U.iloc[:,1]

    tau, _ = kendalltau(x, y)
    rho, _ = spearmanr(x, y)

    return tau, rho

# --------------------------------------------------------
# 2. Ajuste de copulas con AIC/BIC
# --------------------------------------------------------
import numpy as np
from scipy.stats import kendalltau
from statsmodels.distributions.copula.api import GaussianCopula

def ajustar_copulas_bivariadas(U):

    # Convertimos a matrices numpy
    u1 = U.iloc[:,0].to_numpy()
    u2 = U.iloc[:,1].to_numpy()
    U_arr = np.column_stack([u1, u2])
    n = len(U_arr)

    # Medidas empíricas
    tau, _ = kendalltau(u1, u2)
    rho_hat = np.corrcoef(u1, u2)[0,1]

    resultados = []

    # =============== GAUSSIAN COPULA =====================
    try:
        cop = GaussianCopula()

        # El método de statsmodels devuelve un float
        # pero la cópula requiere una matriz 2x2
        R = np.array([[1.0, rho_hat],
                      [rho_hat, 1.0]])

        # Calcular log-likelihood
        ll = np.sum(cop.logpdf(U_arr, R))

        k = 1  # número de parámetros de dependencia
        aic = -2*ll + 2*k
        bic = -2*ll + k*np.log(n)

        resultados.append(("Gaussian", ll, aic, bic, cop, R))

    except Exception as e:
        print("[Advertencia] Error ajustando Gaussiana:", e)

    # OMITIMOS GUMBEL Y T-STUDENT (incompletas)
    print("[Info] Gumbel y t-Student omitidas (statsmodels incompleto).")

    if len(resultados) == 0:
        raise ValueError("No se pudo ajustar ninguna cópula usable.")

    # Elegimos por AIC
    mejor = sorted(resultados, key=lambda x: x[2])[0]

    # Resumen limpio
    resumen = [
        {"familia": r[0], "loglik": float(r[1]), "AIC": float(r[2]), "BIC": float(r[3])}
        for r in resultados
    ]

    return resumen, {
        "familia": mejor[0],
        "copula": mejor[4],
        "corr": mejor[5]
    }



In [20]:
tau, rho = dependencia_empirica(U)
print("Tau:", tau, " — Rho:", rho)

# 3. Ajustar cópulas
resumen, mejor = ajustar_copulas_bivariadas(U)
print("Resumen:", resumen)
print("Mejor cópula:", mejor[0])

U_sim = mejor["copula"].simulate(50000, params=mejor["params"])


Tau: -0.00041708170817081705  — Rho: -0.00045043730850437313
[Advertencia] Error ajustando Gaussiana: EllipticalCopula.pdf() takes from 2 to 3 positional arguments but 4 were given
[Info] Gumbel y t-Student omitidas (statsmodels incompleto).


ValueError: No se pudo ajustar ninguna cópula usable.

In [ ]:


# 4. Simulación
U_sim = mejor[4].random(50000)

Tau: -0.00041708170817081705  — Rho: -0.00045043730850437313
[Advertencia] t-Student falló: 'StudentTCopula' object has no attribute 'fit_df_param'
[Advertencia] Gumbel falló: TransfGumbel.evaluate() missing 1 required positional argument: 'theta'
Resumen: [{'familia': 'Gaussian', 'loglik': np.float64(-5.760947274779939e-13), 'AIC': np.float64(2.0000000000011524), 'BIC': np.float64(9.210340371977336)}]
Mejor cópula: Gaussian


AttributeError: 'GaussianCopula' object has no attribute 'random'